# Data Exploration

This notebook will primarily be to explore the dataset, specifically files under the ``MERGED_CSV`` folder.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import os


In [ ]:
def inspect_file(file_path):
    df = pd.read_csv(file_path, low_memory=False)
    print(f"\n📄 {file_path}")
    print(f"Shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Nulls:\n{df.isnull().sum().sum()}")
    print(f"Data Types:\n{df.dtypes}")
    print(f"First 3 rows:\n{df.head(3)}")
    
    print(df.info)
    return df


In [ ]:
# path = "./dataset/ciciot2023/MERGED_CSV/Merged01.csv"
# df = pd.read_csv(path)
# df.info


In [ ]:
normal_path = "./dataset/edge-iiotset/normal/"
attack_path = "./dataset/edge-iiotset/attack/"
# output_path = "./dataset/edge-iiotset/eda_output/"



In [ ]:
for file in os.listdir(normal_path):
    if file.endswith(".csv"):
        file_path = os.path.join(normal_path, file)
        inspect_file(file_path)
        
for file in os.listdir(attack_path):
    if file.endswith(".csv"):
        file_path = os.path.join(attack_path, file)
        inspect_file(file_path)


## Comment

- None of the attacks seem to have any null values, but many values just have 0
- 63 columns, few may or may not be useful such as time
- Inproportionate attack data
  - Overwhelming amount of DDoS attack data 
    - 229022 in HTTP flood
    - 2914354 in ICMP flood
    - 2020120 in TCP Syn flood
    - 3201626 in UDP flood
  - MITM 1229
  - Backdoor 24862
  - OS fingerprinting 1001
  - Password 1053385
  - Port scanning 22564
  - Ransomware 10925
  - SQL injection 51203
  - Uploading 37634
  - Vuln scan 145869
  - XSS 15915
